In [17]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from os.path import join as oj
from copy import deepcopy
import pandas as pd
import numpy.random as npr
import time
sys.path.append('../../src')
sys.path.append('../../interp')
import utils, lcp, train
from scipy.stats import ttest_ind, spearmanr
from typing import Dict
from compare_stats import compare_stats
import gen_data

# sklearn models
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import export_graphviz, DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.metrics import auc
from all_scores import get_scores

cred = (234/255, 51/255, 86/255)
cblue = (57/255, 138/255, 242/255)
out_dir = '../../results/interp_sim'
os.makedirs(out_dir, exist_ok=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# generate data

In [18]:
n = 100
d = 10

beta = np.zeros(d)
beta[0] = 1
beta[1] = 2
# beta[important_vars] = 1
X, y, _ = gen_data.gen_data(n=n, d=d, beta=beta, var_eps=0.5, 
             s=None, seed=1, shift_type='None', shift_val=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y) # defaults to 0.75: 0.25 split

# fit a model to it

In [20]:
m = train.regress(X, y, model_type='linear')

In [21]:
scores = get_scores(m, X_train, X_test[0])

/accounts/projects/vision/.local/lib/python3.7/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \


In [22]:
pd.DataFrame(scores).style.background_gradient(cmap='viridis')

,shap,lime,ice-contrib,ice-sensitivity
0,1.69235,3.49755,1.72624,1.0513
1,1.767,1.9359,2.09622,2.12779
2,0.0364153,0.140931,0.0242961,-0.0393479
3,0.019868,-0.116486,0.0412768,-0.0773394
4,-0.000135681,0.108711,-0.000284177,0.000530738
5,-0.0841483,0.0862501,-0.0919703,0.0561334
6,0.0689147,0.0539938,0.0699242,0.0748994
7,0.0415624,-0.0465102,0.0609753,0.0430812
8,0.0197551,0.0388597,0.0234699,0.0216637
9,0.0319245,-0.0294201,0.0166922,-0.0656815


In [24]:
ds = compare_stats(beta, scores)


In [40]:
ind_tuples = [(outerKey, innerKey) for outerKey, innerDict in sorted(ds.items()) for innerKey, values in sorted(innerDict.items())]
ind = pd.MultiIndex.from_tuples(ind_tuples, names=['score', 'metric'])
vals = np.array([values for outerKey, innerDict in sorted(ds.items()) for innerKey, values in sorted(innerDict.items())])
vals = vals.reshape(-1, vals.size)

"\n>>> reform\n{('A', 'a'): [1, 2, 3, 4, 5],\n ('A', 'b'): [6, 7, 8, 9, 1],\n ('B', 'a'): [2, 3, 4, 5, 6],\n ('B', 'b'): [7, 8, 9, 1, 2]}\n>>> pandas.DataFrame(reform)\n   A     B   \n"

In [52]:
tab = pd.DataFrame(vals, columns=ind, index=['dset1'])

In [53]:
tab

score         ice-contrib                             ice-sensitivity  \
metric frac_correct_signs frac_intersect rank_corr frac_correct_signs   
dset1                 1.0            1.0       1.0                1.0   

score                                         lime                           \
metric frac_intersect rank_corr frac_correct_signs frac_intersect rank_corr   
dset1             1.0       1.0                1.0            1.0      -1.0   

score                shap                           
metric frac_correct_signs frac_intersect rank_corr  
dset1                 1.0            1.0       1.0

In [67]:
s = tab.transpose().to_latex(multicolumn_format='c')
s = s.replace('\\toprule', '').replace('\\bottomrule', '')
print(s)